## Unzip all inspection reports

In [1]:
import os, zipfile
import glob

path = 'D:/001_Projects/01 - GIPTN/02 - Application development/03 - Bus Inspection/01 - Inspections/July 2018 Bus inspection reports/'
extension = ".zip"


In [6]:
zip_files = glob.glob(path + '/*.zip')

for zip_filename in zip_files:
    dir_name = os.path.splitext(zip_filename)[0]
    os.mkdir(dir_name)
    zip_handler = zipfile.ZipFile(zip_filename, "r")
    zip_handler.extractall(dir_name)

# Read Inspection data


In [2]:
import pandas as pd
import os.path
import re
path = 'D:/001_Projects/01 - GIPTN/02 - Application development/03 - Bus Inspection/01 - Inspections/July 2018 Bus inspection reports/Extracted Reports/'

In [3]:
file_list = []
for root, dirs, files in os.walk(path):
    for filename in files:
        if filename.endswith('.csv'):
            file_list.append(os.path.join(root, filename)) 
    df_list = [pd.read_csv(file, index_col = 'Survey ID', usecols = range(0, 161), nrows=1) for file in file_list]
    df_list_bd = [pd.read_csv(file, usecols = range(0, 172)) for file in file_list]
if df_list:
    inspection_data = pd.concat(df_list, axis=0, sort=False) 
    bd_data = pd.concat(df_list_bd, axis=0, sort=False) 

## Adjust column names, filter and restructure inspection data

In [4]:
inspection_df = inspection_data.drop(['Section Name', 'Vehicle Class', 
                            'Section Name.1', 'Exterior Inspection',
                            'Section Name.3', 'Interior Inspection', 
                            'Section Name.2', 'Vehicle Lights Inspection',
                            'When does the vehicles roadworthy expire?'], axis=1)
inspection_df = inspection_df[inspection_df.columns.drop(list(inspection_df.filter(regex=' - No')))]
inspection_df = inspection_df[inspection_df.columns.drop(list(inspection_df.filter(regex='Image')))]
inspection_df = inspection_df[inspection_df.columns.drop(list(inspection_df.filter(regex='Photo')))]
inspection_df = inspection_df[inspection_df.columns.drop(list(inspection_df.filter(regex='Details')))]
inspection_df = inspection_df[inspection_df.columns.drop(list(inspection_df.filter(regex='details')))]
inspection_df = inspection_df[inspection_df.columns.drop(list(inspection_df.filter(regex='Date by')))]
inspection_df.columns = inspection_df.columns.str.replace("Are ", "")
inspection_df.columns = inspection_df.columns.str.replace("there ", "")
inspection_df.columns = inspection_df.columns.str.replace("any  ", "")
inspection_df.columns = inspection_df.columns.str.replace("Is ", "")
inspection_df.columns = inspection_df.columns.str.replace("the ", "")
inspection_df.columns = inspection_df.columns.str.replace("any ", "")
inspection_df.columns = inspection_df.columns.str.replace(".1", "")
inspection_df.columns = inspection_df.columns.str.replace(" - Yes", "")
inspection_df.columns = inspection_df.columns.str.replace("?", "")
inspection_defects = pd.melt(inspection_df, id_vars=['Survey Name', 'Date and Time of Survey',
   'Vehicle Class (Value)', 'Odometer Reading'], var_name= 'Defect', value_name ='Present')
inspection_defects = inspection_defects.fillna('')
inspection_defects_filtered = inspection_defects[inspection_defects['Present'] == 'Yes']

In [5]:
inspection_defects_filtered.columns

Index(['Survey Name', 'Date and Time of Survey', 'Vehicle Class (Value)',
       'Odometer Reading', 'Defect', 'Present'],
      dtype='object')

## Restructure Body Damage data


In [6]:
bd_filtered = bd_data[pd.notnull(bd_data['Nature of Damage'])]
bd_filled = bd_filtered.fillna(method='ffill')
bd_filled = bd_filled.set_index('Survey ID')
bd_filled = bd_filled.filter(items=['Survey Name', 'Date and Time of Survey', 'Vehicle Class (Value)', 'Last Modified By', 'Last Modified At', 'Nature of Damage'])
bd_filled

,Survey Name,Date and Time of Survey,Vehicle Class (Value),Last Modified By,Last Modified At,Nature of Damage
Survey ID,,,,,,
5e2a9b76-63e8-715d-9fd0-02fad3d2950f,Bus 100,"4 Aug 2018, 14:58",Standard Bus,bevan.reisenberg@aurecongroup.com,8/4/2018 1:08:42 PM,Decal Damage
5e2a9b76-63e8-715d-9fd0-02fad3d2950f,Bus 100,"4 Aug 2018, 14:58",Standard Bus,bevan.reisenberg@aurecongroup.com,8/4/2018 1:06:56 PM,Paint damage
5e2a9b76-63e8-715d-9fd0-02fad3d2950f,Bus 100,"4 Aug 2018, 14:58",Standard Bus,bevan.reisenberg@aurecongroup.com,8/4/2018 1:09:18 PM,Paint damage
5e2a9b76-63e8-715d-9fd0-02fad3d2950f,Bus 100,"4 Aug 2018, 14:58",Standard Bus,bevan.reisenberg@aurecongroup.com,8/4/2018 1:08:06 PM,Paint damage
6f73896f-55a9-04e3-0d86-5a27ff2e0301,Bus 101,"31 Jul 2018, 11:15",Standard Bus,bevan.reisenberg@aurecongroup.com,7/31/2018 9:25:58 AM,Paint damage
6f73896f-55a9-04e3-0d86-5a27ff2e0301,Bus 101,"31 Jul 2018, 11:15",Standard Bus,bevan.reisenberg@aurecongroup.com,7/31/2018 9:27:34 AM,Other
c9f4548a-c060-567f-484e-b674ff89dfb3,Bus 102,"5 Aug 2018, 14:53",Standard Bus,Katlego.mashishi@aurecongroup.com,8/6/2018 7:44:26 AM,Body Damage
2b7b0bf2-2c6f-81d4-55af-96d216ecdf48,Bus 103,"13 Jul 2018, 09:26",Standard Bus,Katlego.mashishi@aurecongroup.com,8/6/2018 7:44:29 AM,Body Damage
2b7b0bf2-2c6f-81d4-55af-96d216ecdf48,Bus 103,"13 Jul 2018, 09:26",Standard Bus,bevan.reisenberg@aurecongroup.com,8/5/2018 12:28:21 PM,Body Damage


In [7]:
filename = 'Full List.csv'
path ='D:/001_Projects/01 - GIPTN/02 - Application development/03 - Bus Inspection/02 - Reports/July 2018/%s' % (filename)
inspection_defects.to_csv(path)

filename = 'Defect List2.csv'
path ='D:/001_Projects/01 - GIPTN/02 - Application development/03 - Bus Inspection/02 - Reports/July 2018/%s' % (filename)
inspection_defects_filtered.to_csv(path)

filename = 'Body Damage2.csv'
path ='D:/001_Projects/01 - GIPTN/02 - Application development/03 - Bus Inspection/02 - Reports/July 2018/%s' % (filename)
bd_filled.to_csv(path)